In [32]:
import datacube
import xarray as xr
from joblib import load
import matplotlib.pyplot as plt
from datacube.utils.cog import write_cog

import sys
sys.path.insert(1, '../../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.classification import predict_xr
from dea_tools.dask import create_local_dask_cluster
import odc.algo
import warnings
warnings.filterwarnings("ignore")

In [2]:
create_local_dask_cluster(spare_mem='2Gb')

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 12,Total memory: 13.10 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34605,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 13.10 GiB
Comm: tcp://127.0.0.1:43361,Total threads: 12
Dashboard: http://127.0.0.1:39275/status,Memory: 13.10 GiB
Nanny: tcp://127.0.0.1:36021,


In [17]:
model_path = 'Linear.joblib'

testing_locations = {
    'Margaret River': (15.75, 80.78),
}

buffer = 0.1

dask_chunks = {'x': 1000, 'y': 1000}

results = 'results/'

In [18]:
dc = datacube.Datacube(app='Classify_satellite_data')

In [19]:
model = load(model_path)

In [33]:
def feature_layers(query):

    # Connect to the datacube
    dc = datacube.Datacube(app='custom_feature_layers')

    # Load ls8 geomedian
    ds = dc.load(product='s2a_sen2cor_granule',
                 **query)

    # Calculate some band indices
    # da = calculate_indices(ds,
    #                        index=['NDVI', 'LAI', 'MNDWI'],
    #                        drop=False)
    dataset =  odc.algo.to_f32(ds)

# Calculate the components that make up the NDVI calculation
    band_diff = dataset.B08_10m - dataset.B03_10m
    band_sum = dataset.B11_20m - dataset.B03_10m

    # Calculate NDVI and store it as a measurement in the original dataset
    mvi = band_diff / band_sum

    # # Add Fractional cover percentiles
    # fc = dc.load(product='ga_ls_fc_pc_cyear_3',
    #              measurements=['pv_pc_10', 'pv_pc_50', 'pv_pc_90'], # only the PV band
    #              like=ds.geobox, # will match geomedian extent
    #              time=query.get('time'),  # use time if in query
    #              dask_chunks=query.get('dask_chunks')  # use dask if in query
    #             )

    # Merge results into single dataset
    # result = xr.merge([da, fc], compat='override')

    return mvi

In [34]:
time = ('2023')
zonal_stats = 'median'
return_coords = True

# Set up the inputs for the ODC query
resolution = (-30, 30)
output_crs = 'EPSG:6933'

In [35]:
# Generate a new datacube query object
query = {
    'time': time,
    'resolution': resolution,
    'output_crs': output_crs,
    'dask_chunks': dask_chunks
}


In [36]:
predictions = []

for key, value in testing_locations.items():

    print('Working on: ' + key)

    bounds = {'x': (value[1] - buffer, value[1] + buffer),
    'y': (value[0] + buffer, value[0] - buffer)}

    # Update datacube query

    query.update(bounds)

    # Load data and calculate features
    data = feature_layers(query).squeeze()

    # Predict using the imported model
    predicted = predict_xr(model,
                           data,
                           proba=True,
                           persist=True,
                           clean=True,
                           return_input=True).compute()

    predictions.append(predicted)

Working on: Margaret River


TypeError: tuple indices must be integers or slices, not str